In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import mysql.connector
import json

# Establish a connection to the MySQL database
connection = mysql.connector.connect(
    host='127.0.0.1',
    port=13306,
    user='root',
    password='root',
    database='pyml'
)

# Read the table data using pandas
query = """SELECT vi.hash_id, vi.contents as question_content, JSON_UNQUOTE(JSON_EXTRACT(tags, '$[*].slug')) AS slug FROM viblo_interview vi"""
df_ques = pd.read_sql(query, connection)

query_ans = """SELECT va.hash_id, va.contents, va.question_id FROM viblo_answer va"""
df_answer = pd.read_sql(query_ans, connection)

# Close the database connection
connection.close()


def filter_slug(x):
    if x is None:
        return ''
    t = str(x).replace("b'", "").replace("'", "")
    t1 = json.loads(t)

    return t1
df_ques['slug_filter'] = df_ques['slug'].apply(filter_slug)
print(df_ques['slug_filter'][0])

# source = 'csv'

# if source == 'csv':
#     df_ques = pd.read_csv('./data/viblo_interview_202306161436.csv')
    
#     def filter_slug(t):
#         if t is None:
#             return ''
#         t1 = json.loads(t)
#         # get column name from list t1
#         t2 = [x['name'] for x in t1]
#         return t2

#     df_ques['slug_filter'] = df_ques['tags'].apply(filter_slug)
#     print(df_ques.head(5))
#     df = df_ques[['id', 'contents', 'slug_filter']]

/var/folders/g6/37kt02914kx36yzcbbqfyck00000gn/T/ipykernel_69039/950260037.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_ques = pd.read_sql(query, connection)
/var/folders/g6/37kt02914kx36yzcbbqfyck00000gn/T/ipykernel_69039/950260037.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_answer = pd.read_sql(query_ans, connection)


['javascript']


In [20]:

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

Y = df_ques['slug_filter']
print(Y[100:110])
# Convert the input data to a binary array
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(Y)


print('y_test', Y[100])
t = mlb.inverse_transform(Y[100:101])
print('t', t)
print('class', mlb.classes_)
   

100          [java]
101         [mysql]
102          [java]
103       [general]
104          [java]
105       [general]
106       [general]
107          [java]
108    [javascript]
109    [javascript]
Name: slug_filter, dtype: object
y_test [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
t [('java',)]
class ['agile' 'ai' 'amazon-web-services' 'android-os' 'angularjs'
 'artificial-intelligence' 'asp-net' 'attack-techniques' 'aws-lambda'
 'backend-development' 'big-data' 'business-analyst' 'c' 'c-2' 'c-lang'
 'chatbot' 'clean-code' 'cloud-computing' 'cloud-security'
 'computer-network' 'computer-vision' 'content-creator' 'cryptography'
 'css' 'cyber-security' 'dart' 'data-science'
 'data-structures-and-algorithms' 'database' 'deep-learning'
 'design-pattern' 'devops' 'distribute-system' 'docker'
 'external-communicati

In [47]:
X = df_ques['question_content']

In [48]:
from keras.models import Sequential
from keras import backend as K, initializers, regularizers, constraints, Model
from keras.layers import Embedding, Flatten, MaxPooling1D, Dense, LSTM, Bidirectional, Attention, Layer, Input, Activation, Dropout, SpatialDropout1D
from keras_preprocessing.text import Tokenizer
from keras_preprocessing import sequence
from gensim import corpora, models
from gensim.models import KeyedVectors
import numpy as np
from keras.utils import to_categorical
import tensorflow as tf

w2v_model = KeyedVectors.load_word2vec_format('../model/wiki.vi.model.bin', binary=True)
tokenizer = Tokenizer()

tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

maxlen = 100
X_1 = sequence.pad_sequences(sequences, maxlen=maxlen)

num_classes = Y.shape[1]

In [49]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X_1, Y, test_size=0.2, random_state=42)
print(X_train.shape, Y_train.shape)

(2893, 100) (2893, 109)


In [45]:
embedding_dim = 400
word_index = tokenizer.word_index
num_words = min(len(word_index) + 1, len(w2v_model.index_to_key))
embedding_matrix = np.zeros((num_words, embedding_dim))

print('num_words:', num_words)
for word, i in word_index.items():
    if i >= num_words:
        continue
    if word in w2v_model.index_to_key:
        embedding_matrix[i] = w2v_model.get_vector(word)
model = Sequential()
model.add(Embedding(num_words, embedding_dim, input_length=maxlen, weights=[embedding_matrix], trainable=False))
model.add(Bidirectional(LSTM(64, return_sequences=True, input_shape=(maxlen, ))))
model.add(Flatten())
model.add(Dense(num_classes, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()
model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_split=0.2)


num_words: 3497
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 100, 400)          1398800   
                                                                 
 bidirectional_4 (Bidirectio  (None, 100, 128)         238080    
 nal)                                                            
                                                                 
 flatten_4 (Flatten)         (None, 12800)             0         
                                                                 
 dense_4 (Dense)             (None, 109)               1395309   
                                                                 
Total params: 3,032,189
Trainable params: 1,633,389
Non-trainable params: 1,398,800
_________________________________________________________________
Epoch 1/10
73/73 [==============================] - 6s 63ms/step - loss: 0.1025 - acc:

In [51]:
from sklearn.metrics import hamming_loss, f1_score

Y_pred = model.predict(X_test)
print('Y_pred', Y_pred.shape, Y_test.shape)

max_indices = np.argmax(Y_pred, axis=1)
# Convert the index to a binary array
Y_pred_binary = np.eye(Y_pred.shape[1])[max_indices]
# print('bin_arr', Y_pred_binary[0])
print('Hamming Loss: ', round(hamming_loss(Y_test, Y_pred_binary),2))
# Compute the F1-score
f1_score = f1_score(Y_test, Y_pred_binary, average='micro')
print('F1-score: ', round(f1_score, 2))


23/23 [==============================] - 0s 19ms/step
Y_pred (724, 109) (724, 109)
Hamming Loss:  0.01
F1-score:  0.28


In [53]:
X_test = [
    'Phân biệt mạng Object Detection 2 stage và 1 stage',
    'Những lỗ hổng thường gặp với API',
    'IAM là gì, IAM có vai trò gì trong việc đảm bảo an toàn hệ thống AWS',
    'Có bao nhiêu loại lỗi sai trong PHP?',
    'Git Submodule trong trường hợp nào ?',
    'Có sự thừa kế theo cấp bậc giữa các module trong không? Giải thích.',
    'Sự khác biệt giữa từ khóa break và continue trong Java?',
    'Trình bày về Output Buffering trong PHP?',
    'RESTful API là gì?',
    'Viết tắt của php có nghĩa là gì ?',
    'Phân biệt POST và GET trong php?',
    'Cờ HttpOnly có tác dụng gì cho cookie?',
    'so sánh sự khác nhau giữa Mysql và MongoDB',
    'Tại sao phải sử dụng hàm khởi tạo?',
    'Cơ chế hoạt động của API ?',
    'Sự khác nhau SOAP và REST APIs?',
]

sequences = tokenizer.texts_to_sequences(X_test)
maxlen = 100
X_seq = sequence.pad_sequences(sequences, maxlen=maxlen)
res_pred = model.predict(X_seq)
print('res_pred', len(res_pred[1]))
max_indices = np.argmax(res_pred, axis=1)
print('max_indices', max_indices)

labels_pred = list(map(lambda x: mlb.classes_[x], max_indices.tolist()))
print('labels_pred', labels_pred)

for index, tags in enumerate(labels_pred):
    print(X_test[index][:100], '->', tags)

1/1 [==============================] - 0s 115ms/step
res_pred 109
max_indices [ 70  24  18  74 103  12  12  74  74  74  74  65  74  12  71 103]
labels_pred ['object-detection', 'cyber-security', 'cloud-security', 'php', 'vue-js', 'c', 'c', 'php', 'php', 'php', 'php', 'network-security', 'php', 'c', 'object-oriented-programming', 'vue-js']
Phân biệt mạng Object Detection 2 stage và 1 stage -> object-detection
Những lỗ hổng thường gặp với API -> cyber-security
IAM là gì, IAM có vai trò gì trong việc đảm bảo an toàn hệ thống AWS -> cloud-security
Có bao nhiêu loại lỗi sai trong PHP? -> php
Git Submodule trong trường hợp nào ? -> vue-js
Có sự thừa kế theo cấp bậc giữa các module trong không? Giải thích. -> c
Sự khác biệt giữa từ khóa break và continue trong Java? -> c
Trình bày về Output Buffering trong PHP? -> php
RESTful API là gì? -> php
Viết tắt của php có nghĩa là gì ? -> php
Phân biệt POST và GET trong php? -> php
Cờ HttpOnly có tác dụng gì cho cookie? -> network-security
so sánh sự 

# Kết luận.
- Model dùng word2vec pretrained (wiki.vi.model.bin) , giúp tính toán việc tạo tag khá ổn.
- F1 score = 0.28, hơi thấp, nhưng do dữ liệu train còn ít (chỉ khoảng 3k câu) nên chưa thể đánh giá được.